In [ ]:
!pip install azure-ai-evaluation[redteam]


Note: Foundry Project should be in these regions:
East US2,
Sweden Central,
France Central,
Switzerland West,

In [ ]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory
import os
from dotenv import load_dotenv

load_dotenv()
# Azure AI Project Information
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        #RiskCategory.HateUnfairness,
        #RiskCategory.Sexual,
        #RiskCategory.SelfHarm
    ], 
    num_objectives=2, # optional, defaults to 10
)

In [ ]:
# Configuration for Azure OpenAI model
azure_openai_config = {
    "azure_endpoint": os.getenv("OAI_ENDPOINT"),
    "azure_deployment": os.getenv("CHAT_MODEL"),
}

red_team_result = await red_team_agent.scan(target=azure_openai_config)

In [ ]:
import os
from typing import Any
from pathlib import Path
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import CodeInterpreterTool

# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
model = os.getenv("CHAT_MODEL")

project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

agent = project_client.agents.get_agent(
    agent_id="asst_QawX7pFenC41tyDAis9ZVTDA"
)


In [ ]:


# Define a simple callback function that simulates a chatbot
def simple_callback(query: str) -> str:
    code_interpreter = CodeInterpreterTool()

    # Create a thread which is a conversation session between an agent and a user.
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=query,
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent to process tne message in the thread
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to increase the token limit
        print(f"Run failed: {run.last_error}")

    # Get all messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    return messages.data[0].content[0].text.value

red_team_result = await red_team_agent.scan(target=simple_callback)
